In [37]:
import pandas as pd
import getpass
from huggingface_hub import notebook_login
import os

from datasets import load_dataset
from fireworks.client import Fireworks
from pydantic import BaseModel, Field
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from openai import OpenAI

In [12]:
notebook_login()

In [13]:
os.environ["FIREWORKS_API_KEY"] = getpass.getpass("fireworks api:")
client = Fireworks(api_key=os.environ["FIREWORKS_API_KEY"])

In [14]:
# sample_data = pd.read_excel('english-danish-openai.xlsx')

# sample_data.to_csv ("english-danish-openai.csv",
#                   index = None,
#                   header=True)

english_danish_sample_data = pd.read_csv('english-danish-openai.csv')
english_danish_sample_data

,English,Danish
0,The cat sat on the windowsill watching the bi...,Katten sad på vindueskarmen og kiggede på fugl...
1,She baked a delicious chocolate cake for her f...,Hun bagte en lækker chokoladekage til sin vens...
2,The new park in the city center has become a p...,Den nye park i byens centrum er blevet et popu...
3,He quickly realized that learning a new langua...,"Han indså hurtigt, at det at lære et nyt sprog..."
4,The old castle on the hill offers a beautiful ...,Det gamle slot på bakken tilbyder en smuk udsi...
...,...,...
114,He enjoyed listening to classical music while ...,"Han nød at lytte til klassisk musik, mens han ..."
115,The old library was filled with countless trea...,Det gamle bibliotek var fyldt med utallige ska...
116,She spent her afternoons tending to her garden.,Hun tilbragte sine eftermiddage med at passe s...
117,The children's laughter echoed in the playground.,Børnenes latter genlød på legepladsen.


In [15]:
instruction = """
          you are an expert translator between English and Danish

          #user will provide you sentences in English 
          #translate users english sentence to Danish.
          #you can only use users english sentence
          
  """

def translate_english_to_danish(english_samples_csv_file, model):
    list_of_danish_sentences = list()
    df = pd.read_csv(english_samples_csv_file)
    for i, row in enumerate(df.iterrows()):
        danish_sentence = client.chat.completions.create(
            model=model,
            messages=[
              {"role": "system", "content": instruction},
              {"role": "user", "content": row[1]['English']}
            ],
        )
        response = danish_sentence.choices[0].message.content
        list_of_danish_sentences.append(response)    
    return list_of_danish_sentences

    


In [ ]:
llama_8b_translation = translate_english_to_danish('./english-danish-openai.csv','accounts/fireworks/models/llama-v3-8b-instruct')

print(llama_8b_translation[0])

Katten sad på vindueskarmen og betragtede fuglene udenfor.


In [25]:
# english_danish_sample_data["Danish_llama_3"] = llama_8b_translation

english_danish_sample_data

,English,Danish,Danish_llama_3
0,The cat sat on the windowsill watching the bi...,Katten sad på vindueskarmen og kiggede på fugl...,Katten sad på vindueskarmen og betragtede fugl...
1,She baked a delicious chocolate cake for her f...,Hun bagte en lækker chokoladekage til sin vens...,Hun bagede en smuk chokoladekage til sin venin...
2,The new park in the city center has become a p...,Den nye park i byens centrum er blevet et popu...,Nyt park i byens centrum er blevet en populær ...
3,He quickly realized that learning a new langua...,"Han indså hurtigt, at det at lære et nyt sprog...","Han realiserede hurtigt, at lære et nyt sprog ..."
4,The old castle on the hill offers a beautiful ...,Det gamle slot på bakken tilbyder en smuk udsi...,Den gamle borg på bakken tilbyder et smukt uds...
...,...,...,...
114,He enjoyed listening to classical music while ...,"Han nød at lytte til klassisk musik, mens han ...","Han nyttede til at lytte til klassisk musik, m..."
115,The old library was filled with countless trea...,Det gamle bibliotek var fyldt med utallige ska...,Den gamle bibliotek var udfyldt med uhørt mang...
116,She spent her afternoons tending to her garden.,Hun tilbragte sine eftermiddage med at passe s...,Hun brugte sine efteraften på at tage sig om h...
117,The children's laughter echoed in the playground.,Børnenes latter genlød på legepladsen.,Barnets le med i skatepladsen.


In [38]:

openai_client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [53]:
# instruction = """
#         you are an expert translator between English and Danish

#         #user will only give you samples of a sentence translated from English to Danish#
#         # Give the translation a score on a scale from one to ten#
#         #format should be something like 'score 1'
#         #Think through your reasoning step-by-step and write the score at the end.#
          
#   """


instruction = """
        you are an expert translator between English and Danish

        #user will only give you samples of a sentence translated from English to Danish#
        # Give the translation a score on a scale from one to ten#
        #format should only be your score          
  """


def evaluate_danish_sentences(english_danish_open_csv, evaluation_sentences):
    scores = list()
    df = pd.read_csv(english_danish_open_csv)
    df["Danish_llama_3"] = evaluation_sentences

    for i, row in enumerate(df.iterrows()): 
        
        response = openai_client.chat.completions.create(
            messages=[
                {"role": "system", "content": instruction},
                {"role": "user", "content": f'''English:{row[1]['English']} 
                                                Danish:{row[1]['Danish_llama_3']}'''}
            ],
            model="gpt-4o",

        )
        try:
            response = response.choices[0].message.content
            #score = int(json.loads(response.split('\n')[-1])['Score'])  
            scores.append(int(response))
        except json.JSONDecodeError as jde:
            continue

    return sum(scores) / len(scores)


llama_8b_avg_score = evaluate_danish_sentences("./english-danish-openai.csv", llama_8b_translation)

# print(f"Llama3 8B: {round(llama_8b_avg_score, 2)}")
print(llama_8b_avg_score)


6.361344537815126


In [46]:
print(llama_8b_avg_score[0])


1. "The cat" is correctly translated to "Katten."
2. "sat" is correctly translated to "sad."
3. "on the windowsill" is correctly translated to "på vindueskarmen."
4. "watching" is correctly translated to "og betragtede."
5. "the birds outside" is correctly translated to "fuglene udenfor."

All components of the sentence are accurately translated, maintaining both meaning and grammatical structure.

Score: 10
